In [1]:
%cd ~
%cd ../../disk2/jl/jl/data_analyzing_tool
import numpy as np

/home/eis
/disk2/jl/jl/data_analyzing_tool


/home/eis/anaconda3/envs/jl_maa/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/eis/anaconda3/envs/jl_maa/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
%load_ext autoreload
%autoreload 
from data_prepare.data_prepare_manager import select_dataset

dataset_name = "cifar-10"
dataset_A, classes_A = select_dataset(dataset_name)

dataset_name = "cifar-10_jitter"
dataset_B, classes_B = select_dataset(dataset_name)

dataset_name = "cifar-10_rotate"
dataset_C, classes_C = select_dataset(dataset_name)

dataset_name = "cifar-10_noise"
dataset_D, classes_D = select_dataset(dataset_name)

All CIFAR-10 files are present.
Computed mean: [125.30691805 122.95039414 113.86538318]
Computed std: [62.99321928 62.08870764 66.70489964]
All CIFAR-10 files are present.
CIFAR-10 loaded.
All CIFAR-10 files are present.
CIFAR-10 loaded.
All CIFAR-10 files are present.
CIFAR-10 loaded.


In [4]:

from model_structure.cnn_model_2d import CNN2D

classes = classes_A = classes_B = classes_C = classes_D
num_classes = len(classes)
print(num_classes)
# from model_structure.testNN_wo_softmax import testNN_wo_Softmax
# model = testNN_wo_Softmax(outputClasses=num_classes)

from model_structure.testModel_wo_softmax_5_layer import testNN_wo_Softmax_5_layer
model = testNN_wo_Softmax_5_layer(outputClasses=num_classes)
# model = CNN2D(num_classes=9)
# pth_path = './trained_pth/rootModel-da 1 - 0.pth'
# pth_path = './trained_pth/best_model_pretrain-1734930660.pth'

# CLustered Pick
'''
pth_path_list = ['./trained_pth/rootModel-fed_avg - MD Mixed | RP.pth',
                 './trained_pth/rootModel-fed_prox - MD Mixed | RP | Mu 0.001.pth',
                 './trained_pth/rootModel-fisher_client - MD Mixed | RP | Mu 2.0.pth',
                 './trained_pth/rootModel-fisher_server - MD Mixed | RP | Mu 1.0.pth',
                 './trained_pth/rootModel-DA| weighted fed_avg_param_diff - MD Mixed | RP.pth']
'''

'''
pth_path_list = ['./trained_pth/rootModel-fed_avg - MD Mixed | RP.pth',
                 './trained_pth/rootModel-fed_prox - MD Mixed | RP | Mu 0.001.pth',
                 './trained_pth/rootModel-DA| weighted fed_avg_param_diff - MD Mixed | RP.pth']
'''

pth_path_list = ['./trained_pth/rootModel-fed_avg_more_filters - MD Mixed | RP.pth']
# pth_path_list = ['./trained_pth/rootModel-fed_avg_5_layer - MD Mixed | RP.pth']


# './trained_pth/rootModel-fed_avg - SD_original | RP.pth',

10


In [5]:
features = []  # 전역 리스트 (hook_fn에서 사용)

def hook_fn(module, input, output):
    """
    지정된 레이어(예: conv3)의 '입력' 혹은 '출력'을 캡처하는 Hook 함수.
    여기서는 input[0]을 사용하므로, conv3 레이어의 입력 텐서를 수집합니다.
    
    - 어떤 레이어에 걸고 싶은지는 모델 구조에 따라 변경하세요.
    - 혹은 output을 보려면 output을 사용하세요.
    """
    # Hook에 들어오는 input[0]은 (batch_size, channel, height, width) 형태일 가능성이 큽니다.
    # 필요하다면 .detach() 처리 후 numpy()로 변환하세요.
    # GPU->CPU 변환 위해 .cpu() 사용, 미분 그래프에서 분리 위해 .detach() 사용
    batch_features = input[0].detach().cpu().numpy()
    features.append(batch_features)
    
def extract_per_sample_features(model, inference_instance, layer_name):
    """
    주어진 모델과 inference_instance(= 데이터셋 + forward)를 이용해
    전체 데이터셋 각 샘플별 feature를 추출하여 반환.
    """
    global features
    features = []  # 매 호출 시 초기화

    # Hook 등록 (conv3 레이어)
    # hook = model.conv3.register_forward_hook(hook_fn)
    # hook = model.conv1.register_forward_hook(hook_fn)
    
    layer = getattr(model, layer_name, None)
    hook = layer.register_forward_hook(hook_fn)

    # (아래 코드는 예시입니다. 실제론 inference_instance.examin()이
    # 전체 샘플에 대해 forward, loss/acc 계산을 수행한다고 가정)
    avg_loss, acc, all_targets, all_outputs = inference_instance.examin()

    # Hook 해제
    hook.remove()

    # features: 각 배치마다 쌓인 numpy 배열들의 리스트
    # -> shape (num_samples, C, H, W) 로 합치기
    extracted_features = np.concatenate(features, axis=0)

    # 필요에 따라 (num_samples, C*H*W) 형태로 리쉐이프
    B, C, H, W = extracted_features.shape
    extracted_features = extracted_features.reshape(B, -1)

    return extracted_features, all_targets  # (num_samples, feature_dim), (num_samples, )

##########################################
# 2) 샘플별 L2 / Cosine 값 전체를 구하는 함수 #
##########################################

def calculate_per_sample_distances(original_features, compared_features):
    """
    original_features와 compared_features 각각 (num_samples, feature_dim) 형태.
    동일 인덱스끼리 L2/Cosine을 구해 (num_samples,) 형태로 반환.
    """
    eps = 1e-8
    
    # L2 distance
    diff = original_features - compared_features  # (num_samples, feature_dim)
    l2_all = np.sqrt(np.sum(diff**2, axis=1))    # (num_samples,)
    
    # L2 distance normalization
    x_norm = np.sqrt(np.sum(original_features**2, axis=1))  # (num_samples,)
    y_norm = np.sqrt(np.sum(compared_features**2, axis=1))  # (num_samples,)
    l2_normalized = l2_all / (x_norm + y_norm + eps)        # (num_samples,)

    # Cosine similarity
    dot_xy = np.sum(original_features * compared_features, axis=1)
    x_norm = np.sqrt(np.sum(original_features**2, axis=1))
    y_norm = np.sqrt(np.sum(compared_features**2, axis=1))
    cos_all = dot_xy / (x_norm * y_norm + eps)   # (num_samples,)

    return l2_normalized, cos_all

In [6]:
from cka_analysis.cka import *

np.random.seed(42)
X = np.random.rand(100, 50).astype(np.float32)  # Use consistent dtype
Y = np.random.rand(100, 50).astype(np.float32)  # Use consistent dtype

K1 = linear_kernel(X)
K2 = linear_kernel(Y)

cka_value = cka(K1, K2)
print(f"CKA value: {cka_value}")

CKA value: 0.3420460522174835
